## 0. Setup

In [1]:
import os
import json
from pathlib import Path
from typing import List, Dict, Any
from datetime import datetime

# LangChain core
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Vector store (usando la versión community en lugar de langchain_chroma)
from langchain_community.vectorstores import Chroma

# Loaders
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredFileLoader

# OpenAI
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [2]:
# Configuración de la API Key de OpenAI
import os

# 🔑 Opción 1: Defínela manualmente aquí (no recomendado en notebooks públicos)
# os.environ["OPENAI_API_KEY"] = "tu_api_key_aqui"

# 🔑 Opción 2 (recomendada): si estás en Colab, usa el gestor de secretos
try:
    from google.colab import userdata
    openai_api_key = userdata.get("OPENAI_API_KEY")
    os.environ["OPENAI_API_KEY"] = openai_api_key
except Exception as e:
    # Si no estás en Colab, intenta leer de variable local
    openai_api_key = os.getenv("OPENAI_API_KEY", "")
    if not openai_api_key:
        raise ValueError("❌ No se encontró OPENAI_API_KEY. Defínela antes de continuar.")

print("✅ API Key configurada correctamente")

✅ API Key configurada correctamente


## 1. Extracting files

In [3]:
# Definir directorios de trabajo

# Carpeta donde estarán los documentos (definida en Celda 4 también)
DATA_DIR = Path("./data")
DATA_DIR.mkdir(exist_ok=True)

# Carpeta donde se guardará la base vectorial Chroma
PERSIST_DIR = Path("./data/chroma")
PERSIST_DIR.mkdir(exist_ok=True, parents=True)

print("✅ Directorios listos")
print("📂 DATA_DIR:", DATA_DIR.resolve())
print("📂 PERSIST_DIR:", PERSIST_DIR.resolve())

✅ Directorios listos
📂 DATA_DIR: /content/data
📂 PERSIST_DIR: /content/data/chroma


In [4]:
import os
from pathlib import Path

# Carpeta donde deben estar tus documentos (PDF, DOCX, TXT, MD, HTML)
DATA_DIR = Path("./data")
DATA_DIR.mkdir(exist_ok=True)

print("✅ Entorno listo. Los documentos se tomarán desde:", DATA_DIR.resolve())

# Listar archivos encontrados en ./data
print("📦 Archivos detectados en ./data:")
for p in DATA_DIR.glob("*"):
    if p.suffix.lower() in [".pdf", ".docx", ".doc", ".txt", ".md", ".html", ".htm"]:
        print(" -", p.name)

✅ Entorno listo. Los documentos se tomarán desde: /content/data
📦 Archivos detectados en ./data:


In [6]:
# 🚀 Opción A: clonar tu repo y copiar documentos a ./data

# 1) Clonar tu repositorio desde GitHub
!rm -rf repo
!git clone https://github.com/RocioPortillo86/SoporteTecnico-Rag.git repo

# 2) Crear carpeta ./data si no existe
!mkdir -p data

# 3) Copiar documentos relevantes desde el repo a ./data
!find repo -type f \( -iname "*.pdf" -o -iname "*.docx" -o -iname "*.doc" -o -iname "*.txt" -o -iname "*.md" -o -iname "*.html" -o -iname "*.htm" \) -exec cp -v {} data/ \;

# 4) Verificación visual rápida
!ls -la data | head -n 50

Cloning into 'repo'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 93 (delta 36), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (93/93), 1.79 MiB | 4.71 MiB/s, done.
Resolving deltas: 100% (36/36), done.
'repo/data/Guía de problemas frecuentes con equipo de cómputo.pdf' -> 'data/Guía de problemas frecuentes con equipo de cómputo.pdf'
'repo/data/Guia Win10 Win11.pdf' -> 'data/Guia Win10 Win11.pdf'
'repo/data/PR-TI-SP-001 Procedimiento de Recepcion Diagnostico de Equipos de computo.pdf' -> 'data/PR-TI-SP-001 Procedimiento de Recepcion Diagnostico de Equipos de computo.pdf'
'repo/requirements.txt' -> 'data/requirements.txt'
'repo/README.md' -> 'data/README.md'
total 2044
drwxr-xr-x 3 root root    4096 Sep 15 23:38 .
drwxr-xr-x 1 root root    4096 Sep 15 23:38 ..
drwxr-xr-x 2 root root    4096 Sep 15 23:32 chroma
-rw-r--r-- 1 root root   64818 Sep 15 23:38 Guía 

In [7]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredFileLoader
from langchain.docstore.document import Document

folder_path = str(DATA_DIR)  # <-- usamos la carpeta definida en la celda 4

# Buscar archivos comunes (recursivo)
patterns = ["**/*.pdf", "**/*.txt", "**/*.md", "**/*.docx", "**/*.html", "**/*.htm"]
file_paths = []
for p in patterns:
    file_paths.extend(glob(os.path.join(folder_path, p), recursive=True))

if not file_paths:
    raise FileNotFoundError(
        f"No se encontraron archivos en {folder_path}. "
        "Coloca tus documentos en ./data."
    )

docs = []
for path in file_paths:
    try:
        low = path.lower()
        if low.endswith(".pdf"):
            loader = PyPDFLoader(path)
        elif low.endswith(".txt") or low.endswith(".md"):
            loader = TextLoader(path)
        else:
            loader = UnstructuredFileLoader(path)  # Para DOCX/HTML
        docs.extend(loader.load())
    except Exception as e:
        print(f"⚠️ Error cargando {os.path.basename(path)}: {e}")

print(f"✅ Se cargaron {len(docs)} fragmentos desde {len(file_paths)} archivos")

✅ Se cargaron 30 fragmentos desde 5 archivos


## 2. Text Splitting into Chunks

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Definir parámetros de fragmentación
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

# Dividir los documentos cargados en fragmentos
split_docs = text_splitter.split_documents(docs)

print(f"✅ Se crearon {len(split_docs)} fragmentos (chunk_size={CHUNK_SIZE}, overlap={CHUNK_OVERLAP})")

# Mostrar vista previa del primer fragmento
if split_docs:
    print("Ejemplo de fragmento:")
    print("Archivo:", split_docs[0].metadata.get("filename", "desconocido"))
    print("Texto:", split_docs[0].page_content[:200].replace("\n", " "), "...")

✅ Se crearon 51 fragmentos (chunk_size=1000, overlap=200)
Ejemplo de fragmento:
Archivo: desconocido
Texto: Guía rapida de problemas frecuentes con equipo de cómputo  Fuente de alimentación  Problema: El ordenador se apaga de repente sin razón y hace mucho ruido  Solución: Limpiar la fuente de poder por den ...


## 3. Embedding

In [9]:
from langchain_openai import OpenAIEmbeddings

# Crear embeddings con el modelo moderno (más rápido y económico que ada-002)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=openai_api_key   # asegúrate que tu API Key está cargada en la celda 1
)

print("✅ Embeddings listos con text-embedding-3-small")

✅ Embeddings listos con text-embedding-3-small


## 4. Vector Stores

In [10]:
from langchain_chroma import Chroma
from pathlib import Path

# Carpeta donde se guardará la base vectorial
PERSIST_DIR = Path("./data/chroma")
PERSIST_DIR.mkdir(exist_ok=True, parents=True)

# Crear y persistir el vector store con los fragmentos y los embeddings ya definidos
vectordb = Chroma.from_documents(
    documents=split_docs,      # viene de la celda 8
    embedding=embeddings,      # viene de la celda 10
    persist_directory=str(PERSIST_DIR)
)


print(f"✅ Vector store creado y guardado en: {PERSIST_DIR.resolve()}")

✅ Vector store creado y guardado en: /content/data/chroma


## 5. Retriving from the Persistant Vector Datastore

In [11]:
from langchain_chroma import Chroma

# Reabrir Chroma desde disco usando los mismos embeddings
vectordb = Chroma(
    persist_directory=str(PERSIST_DIR),
    embedding_function=embeddings
)

# Preparar el retriever (k=4 por defecto)
retriever = vectordb.as_retriever(search_kwargs={"k": 4})

print("✅ Chroma reabierto desde:", PERSIST_DIR.resolve())
print("🔎 Retriever listo (k=4)")

✅ Chroma reabierto desde: /content/data/chroma
🔎 Retriever listo (k=4)


## 6. Retrivers in Langchain

In [ ]:
# --- Autorecuperación de qa si no existe ---
import os

def _ensure_qa():
    global qa
    try:
        qa  # ya existe
        return
    except NameError:
        pass  # no existe, lo creamos

    # Imports necesarios
    from langchain_openai import ChatOpenAI, OpenAIEmbeddings
    try:
        # preferible si NO tienes langchain-chroma instalado
        from langchain_community.vectorstores import Chroma
    except Exception:
        # alternativa si usas el paquete separado
        from langchain_chroma import Chroma
    from langchain.chains import RetrievalQA

    # Parámetros desde env (útiles en GitHub Actions); pon por defecto si no hay env
    PERSIST_DIR = os.getenv("PERSIST_DIR", "data/chroma")
    EMB_MODEL   = os.getenv("MODEL_EMB", "text-embedding-3-small")
    LLM_MODEL   = os.getenv("LLM_MODEL", "gpt-4o-mini")
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    assert OPENAI_API_KEY, "Falta OPENAI_API_KEY"

    # Reabrir base y crear retriever + chain
    embeddings = OpenAIEmbeddings(model=EMB_MODEL, api_key=OPENAI_API_KEY)
    vectordb   = Chroma(persist_directory=PERSIST_DIR, embedding_function=embeddings)
    retriever  = vectordb.as_retriever(search_kwargs={"k": 4})

    llm = ChatOpenAI(model=LLM_MODEL, temperature=0, api_key=OPENAI_API_KEY)
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True,
        chain_type="stuff"
    )

_ensure_qa()


from pathlib import Path

preguntas = [
    "La computadora se apaga de repente, ¿qué reviso?",
    "La impresora atasca las hojas, ¿cómo lo soluciono?",
    "¿Cómo restablecer la conexión a internet si no funciona?",
    "¿Qué hacer si la computadora hace mucho ruido?",
    "¿Qué información principal contienen estos documentos?"
]

def _cite(meta: dict) -> str:
    page = meta.get("page") or meta.get("page_number") or meta.get("source_page") or "?"
    fn = Path(meta.get("filename", meta.get("source","doc"))).name
    return f"[{fn}:{page}]"

result_rows = []
for i, q in enumerate(preguntas, 1):
    out = qa.invoke({"query": q})
    answer  = out.get("result","")
    sources = out.get("source_documents", []) or []
    if "_Citas_:" not in answer and sources:
        cites = " ".join(_cite(d.metadata) for d in sources[:3])
        answer += f"\n\n_Citas_: {cites}"
    result_rows.append({"#": i, "pregunta": q, "respuesta": answer})

print(f"✅ Generadas {len(result_rows)} respuestas")



❓ Pregunta 1: La computadora se apaga de repente, ¿qué reviso?
**Diagnóstico breve**
- La computadora puede estar experimentando sobrecalentamiento de la fuente de alimentación o desgaste de la misma.

**Pasos para resolver**
1. Limpiar la fuente de poder por dentro para eliminar el polvo que puede estar causando el sobrecalentamiento.
2. Verificar si la parte trasera de la fuente está sobrecalentada y mantener las tomas de aire limpias.
3. Si el problema persiste, considerar el desgaste de la fuente y evaluar la posibilidad de cambiarla.

**Verificación**
- Asegurarse de que la computadora encienda correctamente después de realizar la limpieza y verificar que no se apague de nuevo.

**Notas / Advertencias**
- Si la limpieza no resuelve el problema, puede ser necesario reemplazar la fuente de alimentación.

_Citas_: [archivo:página] [archivo:página]

❓ Pregunta 2: La impresora atasca las hojas, ¿cómo lo soluciono?
**Diagnóstico breve**
- El papel se traba siempre en la impresora.

**P

In [ ]:
# ✅ 5 preguntas → guarda MD/CSV en carpeta results/ (sin git push aquí)

from pathlib import Path
from datetime import datetime
import pandas as pd

preguntas = [
    "La computadora se apaga de repente, ¿qué reviso?",
    "La impresora atasca las hojas, ¿cómo lo soluciono?",
    "¿Cómo restablecer la conexión a internet si no funciona?",
    "¿Qué hacer si la computadora hace mucho ruido?",
    "¿Qué información principal contienen estos documentos?"
]

# --- hacer preguntas con qa ---
from pathlib import Path as _P
def _cite(meta: dict) -> str:
    page = meta.get("page") or meta.get("page_number") or meta.get("source_page") or "?"
    fn = _P(meta.get("filename", meta.get("source","doc"))).name
    return f"[{fn}:{page}]"

assert "qa" in globals(), "Falta el objeto `qa` antes de esta celda."

result_rows = []
for i, q in enumerate(preguntas, 1):
    out = qa.invoke({"query": q})
    answer  = out.get("result","")
    sources = out.get("source_documents", []) or []
    if "_Citas_:" not in answer and sources:
        cites = " ".join(_cite(d.metadata) for d in sources[:3])
        answer += f"\n\n_Citas_: {cites}"
    result_rows.append({"#": i, "pregunta": q, "respuesta": answer})

# --- guardar en results/ ---
now = datetime.now().strftime("%Y%m%d_%H%M")
results_dir = Path("results")
results_dir.mkdir(parents=True, exist_ok=True)

md_path  = results_dir / f"QA_results_{now}.md"
csv_path = results_dir / f"QA_results_{now}.csv"

with open(md_path, "w", encoding="utf-8") as f:
    f.write(f"# Resultados QA Soporte Técnico — {now}\n\n")
    for row in result_rows:
        f.write(f"## ❓ Pregunta {row['#']}\n{row['pregunta']}\n\n")
        f.write(f"**Respuesta**\n\n{row['respuesta']}\n\n---\n\n")

pd.DataFrame(result_rows).to_csv(csv_path, index=False, encoding="utf-8")

print("✅ Archivos generados:", md_path, csv_path)


✅ Resultados subidos a https://github.com/RocioPortillo86/SoporteTecnico-Rag/tree/main/results
